In [50]:
open('cafe.txt', 'w', encoding = 'utf8').write('café')

4

In [51]:
open('cafe.txt').read()

'café'

In [52]:
import sys, locale

expressions = """
  locale.getpreferredencoding()
  type(my_file)
  my_file.encoding
  sys.stdout.isatty()
  sys.stdout.encoding
  sys.stdin.isatty()
  sys.stdin.encoding
  sys.stderr.isatty()
  sys.stderr.encoding
  sys.getdefaultencoding()
  sys.getfilesystemencoding()
"""
my_file = open('dummy', 'w')
for expression in expressions.split():
    value = eval(expression)
    print(expression.rjust(30), '->', repr(value))

 locale.getpreferredencoding() -> 'UTF-8'
                 type(my_file) -> <class '_io.TextIOWrapper'>
              my_file.encoding -> 'UTF-8'
           sys.stdout.isatty() -> False
           sys.stdout.encoding -> 'UTF-8'
            sys.stdin.isatty() -> False
            sys.stdin.encoding -> 'utf-8'
           sys.stderr.isatty() -> False
           sys.stderr.encoding -> 'UTF-8'
      sys.getdefaultencoding() -> 'utf-8'
   sys.getfilesystemencoding() -> 'utf-8'


In [53]:
from unicodedata import normalize
len(normalize('NFC',s1)), len(normalize('NFC',s2))

(4, 4)

In [54]:
len(normalize('NFD',s1)),len(normalize('NFD',s2))

(5, 5)

In [55]:
normalize('NFC',s1), normalize('NFC',s2)

('café', 'café')

In [56]:
normalize('NFD',s1), normalize('NFD',s2)

('café', 'café')

In [57]:
normalize('NFC',s1) == normalize('NFC',s2)

True

In [58]:
normalize('NFD',s1) == normalize('NFD',s2)

True

In [59]:
from unicodedata import normalize, name
ohm = '\u2126'
name(ohm)

'OHM SIGN'

In [60]:
ohm_c = normalize('NFC', ohm)
name(ohm_c)

'GREEK CAPITAL LETTER OMEGA'

In [61]:
ohm, ohm_c, ohm == ohm_c

('Ω', 'Ω', False)

In [62]:
normalize('NFC', ohm) == normalize('NFC', ohm_c)

True

In [63]:
eszett = 'ß'
name(eszett)

'LATIN SMALL LETTER SHARP S'

In [64]:
eszett_cf = eszett.casefold()
eszett_cf2 = eszett.lower()
eszett, eszett_cf, eszett_cf2

('ß', 'ss', 'ß')

In [65]:
s1 = 'café'
s2 = 'cafe\u0301'
s1 == s2

False

In [66]:
from unicodedata import normalize

def nfc_equal(str1, str2):
    return normalize('NFC', str1) == normalize('NFC', str2)

def fold_equal(str1, str2):
    return (normalize('NFC', str1).casefold() ==
            normalize('NFC', str2).casefold())

nfc_equal(s1,s2)

True

In [67]:
nfc_equal('A','a')

False

In [68]:
s3 = 'Straße'
s4 = 'strasse'
s3 == s4


False

In [69]:
nfc_equal(s3, s4)

False

In [70]:
fold_equal(s3, s4)

True

In [71]:
fold_equal(s1, s2)

True

In [72]:
fold_equal('A','a')

True

In [73]:
import unicodedata
import string

def shave_marks(txt):
    norm_txt = unicodedata.normalize('NFD', txt)
    shaved = ''.join(c for c in norm_txt
                     if not unicodedata.combining(c))
    return unicodedata.normalize('NFC', shaved)

In [74]:
order = '"Herr Voß 1⁄2", Sir;'
shave_marks(order)

'"Herr Voß 1⁄2", Sir;'

In [75]:
Greek = 'Z mercytania.'
shave_marks(Greek)

'Z mercytania.'

In [76]:
shave_marks('Straße')

'Straße'

In [77]:
shave_marks('café')

'cafe'

In [78]:
def shave_marks_latin(txt):
  """Remove all diacritic marks from Latin base characters."""
  norm_txt = unicodedata.normalize('NFD', txt)
  latin_base = False
  keepers = []

  for c in norm_txt:
    if unicodedata.combining(c) and latin_base:
      continue # ignore diacritic on Latin base char
    keepers.append(c)
    # if it isn't combining char, it's a new base char

    if not unicodedata.combining(c):
      latin_base = c in string.ascii_letters
  shaved = ''.join(keepers)
  return unicodedata.normalize('NFC', shaved)

In [79]:
single_map = str.maketrans("""‚ƒ„†ˆ‹‘’“”•–—˜›""",
                           """'f"*^<''""---~>""")
multi_map = str.maketrans({
  '€': '<euro>',
  '…': '...',
  'Œ': 'OE',
  '™': '(TM)',
  'œ': 'oe',
  '‰': '<per mille>',
  '‡': '**',
})
multi_map.update(single_map)

def dewinize(txt):
  """Replace Win1252 symbols with ASCII chars or sequences."""
  return txt.translate(multi_map)

def asciize(txt):
  no_marks = shave_marks_latin(dewinize(txt))
  no_marks = no_marks.replace('ß', 'ss')
  return unicodedata.normalize('NFKC', no_marks)

In [80]:
order = '"Herr Voß :1/2 cup of '
dewinize(order)

'"Herr Voß :1/2 cup of '

In [81]:
fruits = ['caju', 'atemoia', 'cajá', 'açaí', 'acerola']

sorted(fruits)

['acerola', 'atemoia', 'açaí', 'caju', 'cajá']

In [82]:
!pip install pyuca

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 14.2 MB/s eta 0:00:00


In [83]:
import pyuca
coll = pyuca.Collator()
fruits = ['caju', 'atemoia', 'cajá', 'açaí', 'acerola']
sorted_fruits = sorted(fruits, key = coll.sort_key)
sorted_fruits

['açaí', 'acerola', 'atemoia', 'cajá', 'caju']

In [84]:
import unicodedata
import re

re_digit = re.compile(r'\d')
sample = '1\xbc\xb2\u0969\u136b\u216b\u2466\u2480\u3285'
for char in sample:
  print('U+%04x' % ord(char),
        char.center(6),
        're_dig' if re_digit.match(char) else '-',
        'isdig' if char.isdigit() else '-',
        'isnum' if char.isnumeric() else '-',
        format(unicodedata.numeric(char), '5.2f'),
        unicodedata.name(char),
        sep='\t')

U+0031	  1   	re_dig	isdig	isnum	 1.00	DIGIT ONE
U+00bc	  ¼   	-	-	isnum	 0.25	VULGAR FRACTION ONE QUARTER
U+00b2	  ²   	-	isdig	isnum	 2.00	SUPERSCRIPT TWO
U+0969	  ३   	re_dig	isdig	isnum	 3.00	DEVANAGARI DIGIT THREE
U+136b	  ፫   	-	isdig	isnum	 3.00	ETHIOPIC DIGIT THREE
U+216b	  Ⅻ   	-	-	isnum	12.00	ROMAN NUMERAL TWELVE
U+2466	  ⑦   	-	isdig	isnum	 7.00	CIRCLED DIGIT SEVEN
U+2480	  ⒀   	-	-	isnum	13.00	PARENTHESIZED NUMBER THIRTEEN
U+3285	  ㊅   	-	-	isnum	 6.00	CIRCLED IDEOGRAPH SIX


In [85]:
import re
re_numbers_str = re.compile(r'\d+')
re_words_str = re.compile(r'\w+')
re_numbers_bytes = re.compile(rb'\d+')
re_words_bytes = re.compile(rb'\w+')

text_str = ("Ramanujan saw \u0be7\u0bed\u0be8\u0bef"
              " as 1729 = 1³ + 12³ = 9³ + 10³.")
text_bytes = text_str.encode('utf_8')
print('Text', repr(text_str), sep='\n  ')
print('Numbers')
print('  str  :', re_numbers_str.findall(text_str))
print('  bytes:', re_numbers_bytes.findall(text_bytes))
print('Words')
print('  str  :', re_words_str.findall(text_str))
print('bytes:', re_words_bytes.findall(text_bytes))

Text
  'Ramanujan saw ௧௭௨௯ as 1729 = 1³ + 12³ = 9³ + 10³.'
Numbers
  str  : ['௧௭௨௯', '1729', '1', '12', '9', '10']
  bytes: [b'1729', b'1', b'12', b'9', b'10']
Words
  str  : ['Ramanujan', 'saw', '௧௭௨௯', 'as', '1729', '1³', '12³', '9³', '10³']
bytes: [b'Ramanujan', b'saw', b'as', b'1729', b'1', b'12', b'9', b'10']


In [86]:
import os
os.listdir('.')

['.config', 'dummy', 'cafe.txt', 'sample_data']

In [87]:
os.listdir(b'.')

[b'.config', b'dummy', b'cafe.txt', b'sample_data']

In [89]:
os.listdir('./fluent_python')

FileNotFoundError: [Errno 2] No such file or directory: b'./fluent_python'

In [90]:
os.listdir(b'./fluent_python')

FileNotFoundError: [Errno 2] No such file or directory: b'./fluent_python'

In [91]:
pi_name_bytes = os.listdir(b'.')[1]
pi_name_str = pi_name_bytes.decode('ascii','surrogateescape')
pi_name_str

'dummy'